In [1]:
# Standard Python modules
import os, sys
import glob
import numpy as np
import pandas as pd
import xarray as xr
import re

# extras
%matplotlib inline

# import personal modules
# Path to modules
sys.path.append('../modules')
# Import my modules
from utils import roundPartial, find_closest_MERRA2_lon
from trajectory import combine_IVT_and_trajectory, combine_arscale_and_trajectory


In [2]:
path_to_data = '/data/projects/Comet/cwp140/' 
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
## load Rutz AR
fname = path_to_data + 'preprocessed/MERRA2/MERRA2_Rutz_US-West_latlon_2020-2024.nc'
ar = xr.open_dataset(fname)

print('Loading ERA5 AR Scale')
fname = path_to_data + 'preprocessed/ARScale_ERA5/ERA5_ARScale_WY2023.nc'
arscale = xr.open_dataset(fname)

## load HUC8 IDs
print('Loading HUC8 IDs')
HUC8_IDs = ['14010001', '14080101'] ## get list of HUC8 IDs
HUC8_IDs = ['14010001'] ## get list of HUC8 IDs

Loading ERA5 AR Scale
Loading HUC8 IDs


In [10]:
def combine_arscale_and_trajectory(ERA5, arscale, ar):
    t = xr.DataArray(ERA5.time.values, dims=['location'], name='time') 

    # create a list of lat/lons that match ERA5 spacing
    x = xr.DataArray(roundPartial(ERA5.lon.values, 0.25), dims=['location'])
    y = xr.DataArray(roundPartial(ERA5.lat.values, 0.25), dims=['location'])

    x = xr.DataArray(ERA5.lon.values, dims=("location"), coords={"lon": x}, name='traj_lons')
    y = xr.DataArray(ERA5.lat.values, dims=("location"), coords={"lat": y}, name='traj_lats')

    # create a new dataset that has the trajectory lat and lons and the closest MERRA2 lat/lons as coords
    z = xr.merge([x, y, t])

    ## Open text file with coordinates of coastal region along N. America West Coast
    textpts_fname = '../data/latlon_coast-modified-ERA5.txt'
    df = pd.read_csv(textpts_fname, header=None, sep=' ', names=['latitude', 'longitude'], engine='python')
    df['longitude'] = df['longitude']*-1

    d = {'lat' : df['latitude'],
        'lon' : df['longitude']}

    txtpts = pd.DataFrame(d)
    txtpts = txtpts.drop_duplicates(subset=['lat', 'lon'])

    ## Now loop through the lat/lon pairs and see where they match
    idx_lst = []
    for i, (x, y) in enumerate(zip(z.lon.values, z.lat.values)):
        for j, (lon, lat) in enumerate(zip(txtpts.lon.values, txtpts.lat.values)):
            ## test if lat/lon pair matches
            result_variable = (x == lon) & (y == lat)

            if (result_variable == True):
                idx = (i, j)
                idx_lst.append(idx)

    if len(idx_lst) > 0:
        ## take first time the trajectory crosses the coast
        idx_lat = txtpts.iloc[idx[1]].lat
        idx_lon = txtpts.iloc[idx[1]].lon

        ## this is the time of the trajectory when it crosses west coast
        time_match = z.sel(location=idx[0]).time.values
        ts = pd.to_datetime(str(time_match)).strftime('%Y-%m-%d %H')
        ERA5 = ERA5.assign(time_match=ts)

        #####################
        ### STRICT METHOD ###
        #####################

        ## Gather arscale of closest grid and time value
        arscale_val = arscale.sel(lat=idx_lat, lon=idx_lon, time=time_match, method='nearest')['rank'].values
        ERA5 = ERA5.assign(ar_scale_strict=arscale_val)

        ## Gather Rutz AR value of closest grid and time value
        ar_val = ar.sel(lat=idx_lat, lon=idx_lon, time=time_match, method='nearest')['AR'].values
        ERA5 = ERA5.assign(ar_strict=ar_val)

        coastal_IVT_val = arscale.sel(lat=idx_lat, lon=idx_lon, time=time_match, method='nearest')['IVT'].values
        ERA5 = ERA5.assign(coastal_IVT_strict=coastal_IVT_val)


        #######################
        ### FLEXIBLE METHOD ###
        #######################

        ## select the 12 hours on each side of the time step
        ## select the surrounding grid points within 1 degree
        sta = time_match - np.timedelta64(12,'h')
        sto = time_match + np.timedelta64(12,'h')

        tmp = arscale.sel(lat=slice(idx_lat-1, idx_lat+1), lon=slice(idx_lon-1, idx_lon+1), time=slice(sta, sto))
        arscale_val = tmp['rank'].max().values

        ## get coastal IVT value
        coastal_IVT_val = tmp['IVT'].max().values

        ## now put those values into the trajectory dataset
        ERA5 = ERA5.assign(ar_scale=arscale_val)
        ERA5 = ERA5.assign(coastal_IVT=coastal_IVT_val)

        ## lets also grab whether rutz et al AR was detected
        tmp1 = ar.sel(lat=slice(idx_lat-1, idx_lat+1), lon=slice(idx_lon-1, idx_lon+1), time=slice(sta, sto))
        ar_val = tmp1.AR.values.max()

        ## assign value to trajectory dataset
        ERA5 = ERA5.assign(ar=ar_val)


    else:
        ## since the trajectory didn't cross the west coast, set ar_scale to nan
        ERA5 = ERA5.assign(ar_scale=np.nan)
        ERA5 = ERA5.assign(ar=np.nan)
        ERA5 = ERA5.assign(coastal_IVT=np.nan)

        ERA5 = ERA5.assign(ar_scale_strict=np.nan)
        ERA5 = ERA5.assign(ar_strict=np.nan)
        ERA5 = ERA5.assign(coastal_IVT_strict=np.nan)
        ERA5 = ERA5.assign(time_match='nan')

    return ERA5

In [11]:
## loop through all HUC8s
ds_lst = []
for i, HUC8_ID in enumerate(HUC8_IDs):
    print(i, HUC8_ID)
    ## load watershed trajectories
    fname = path_to_data + 'preprocessed/UCRB_trajectories/PRISM_HUC8_{0}.nc'.format(HUC8_ID)
    ERA5 = xr.open_dataset(fname)
    ERA5 = ERA5.assign_coords({"lon": ERA5.longitude, "lat": ERA5.latitude, "time": ERA5.time})
    ERA5 = ERA5.drop_vars(["latitude", "longitude"])
    ERA5 = ERA5.sel(start_lev=650., grid='center')

    ds_lst = []
    ## loop through all trajectories for that watershed
    for i, st_date in enumerate(ERA5.start_date.values):
        tmp = ERA5.sel(start_date=st_date)
        
        tmp = combine_arscale_and_trajectory(tmp, arscale, ar)
        
        ds_lst.append(tmp)
        
    ## merge final dataset
    final_ds = xr.concat(ds_lst, dim="start_date")

0 14010001


In [3]:
## load Rutz AR
fname = path_to_data + 'preprocessed/MERRA2/MERRA2_Rutz_US-West.nc'
ar = xr.open_dataset(fname)

## load AR scale
fname = path_to_data + 'preprocessed/MERRA2/MERRA2_ARScale_US-West.nc'
arscale = xr.open_dataset(fname)

## load HUC8 IDs
fname = path_to_data + 'preprocessed/PRISM/PRISM_HUC8_CO.nc'
ds = xr.open_dataset(fname)
HUC8_IDs = ds.HUC8.values ## get list of HUC8 IDs

## loop through all HUC8s
for i, HUC8_ID in enumerate(HUC8_IDs):

    ## load watershed trajectories
    fname = path_to_data + 'preprocessed/ERA5_trajectories/PRISM_HUC8_{0}.nc'.format(HUC8_ID)
    ERA5 = xr.open_dataset(fname)
    ERA5 = ERA5.assign_coords({"lon": ERA5.longitude, "lat": ERA5.latitude, "time": ERA5.time})
    ERA5 = ERA5.drop_vars(["latitude", "longitude"])

    ds_lst = []
    ## loop through all trajectories for that watershed
    for i, st_date in enumerate(ERA5.start_date.values):
        tmp = ERA5.sel(start_date=st_date)
        ## combine IVT data   
        tmp = combine_IVT_and_trajectory(tmp)
        ## add arscale
        tmp = combine_arscale_and_trajectory(tmp, arscale, ar)
        ds_lst.append(tmp)

    ## merge final dataset
    final_ds = xr.concat(ds_lst, dim="start_date")

    out_fname = '/home/dnash/comet_data/preprocessed/ERA5_trajectories/final/PRISM_HUC8_{0}.nc'.format(HUC8_ID)
    final_ds.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')

<xarray.Dataset>
Dimensions:       (index: 72, start_date: 87)
Coordinates:
  * index         (index) int64 0 1 2 3 4 5 6 7 8 ... 63 64 65 66 67 68 69 70 71
    time          (start_date, index) datetime64[ns] ...
  * start_date    (start_date) datetime64[ns] 2000-10-24 ... 2019-05-21
    lon           (start_date, index) float64 ...
    lat           (start_date, index) float64 ...
Data variables:
    level         (start_date, index) float64 ...
    q             (start_date, index) float64 ...
    u             (start_date, index) float64 ...
    v             (start_date, index) float64 ...
    w             (start_date, index) float64 ...
    dq            (start_date, index) float64 ...
    drying_ratio  (start_date, index) float64 ...